In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_excel("ai_dev_assignment_tickets_complex_1000.xls")

In [3]:
# !pip install xlrd
# required for reading excel 

In [4]:
df.head()

,ticket_id,ticket_text,issue_type,urgency_level,product
0,1,Payment issue for my SmartWatch V2. I was unde...,Billing Problem,Medium,SmartWatch V2
1,2,Can you tell me more about the UltraClean Vacu...,General Inquiry,NaN,UltraClean Vacuum
2,3,I ordered SoundWave 300 but got EcoBreeze AC i...,Wrong Item,Medium,SoundWave 300
3,4,Facing installation issue with PhotoSnap Cam. ...,Installation Issue,Low,PhotoSnap Cam
4,5,Order #30903 for Vision LED TV is 13 days late...,Late Delivery,NaN,Vision LED TV


In [5]:
df.shape

(1000, 5)

In [6]:
df.dtypes

ticket_id         int64
ticket_text      object
issue_type       object
urgency_level    object
product          object
dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ticket_id      1000 non-null   int64 
 1   ticket_text    945 non-null    object
 2   issue_type     924 non-null    object
 3   urgency_level  948 non-null    object
 4   product        1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [8]:
df.isnull().sum()

ticket_id         0
ticket_text      55
issue_type       76
urgency_level    52
product           0
dtype: int64

In [10]:
print(df['urgency_level'].value_counts())
print(df['issue_type'].value_counts())

urgency_level
High      330
Medium    319
Low       299
Name: count, dtype: int64
issue_type
Billing Problem       146
General Inquiry       146
Account Access        143
Installation Issue    142
Product Defect        121
Wrong Item            114
Late Delivery         112
Name: count, dtype: int64


In [11]:
df = df.dropna(subset=['ticket_text', 'issue_type', 'urgency_level']).copy() # added copy to prevent SettingWithCopyWarning

In [12]:
# !pip install spacy
print(df['urgency_level'].value_counts())
print(df['issue_type'].value_counts())

urgency_level
High      288
Medium    278
Low       260
Name: count, dtype: int64
issue_type
Account Access        133
Installation Issue    128
Billing Problem       128
General Inquiry       127
Product Defect        110
Wrong Item            103
Late Delivery          97
Name: count, dtype: int64


In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [14]:
# !python -m spacy download en_core_web_sm

In [15]:
def preprocess_txt(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(tokens)


In [16]:
df['ticket_text'][0]

'Payment issue for my SmartWatch V2. I was underbilled for order #29224.'

In [17]:
preprocess_txt(df['ticket_text'][0])

'payment issue smartwatch underbilled order'

In [18]:
df['ticket_text_clean'] = df['ticket_text'].apply(preprocess_txt)

In [19]:
df.sample(10)

,ticket_id,ticket_text,issue_type,urgency_level,product,ticket_text_clean
431,432,Can't log in to my account. Keeps showing erro...,Account Access,Low,SoundWave 300,log account keep show error help delivery expe...
528,529,ProTab X1 is missing. It stopped working after...,Product Defect,High,ProTab X1,protab miss stop work day
23,24,Can't log in to my account. Keeps showing bloc...,Account Access,Low,EcoBreeze AC,log account keep show block help
676,677,Facing installation issue with Vision LED TV. ...,Installation Issue,Low,Vision LED TV,face installation issue vision lead tv setup f...
273,274,Payment issue for my SmartWatch V2. I was unde...,Billing Problem,Medium,SmartWatch V2,payment issue smartwatch underbilled order del...
154,155,Payment issue for my . I was not refunded for ...,Billing Problem,Low,FitRun Treadmill,payment issue refund order contact support get...
945,946,Order #26064 for Vision LED TV is 18 days late...,Late Delivery,Low,Vision LED TV,order vision lead tv day late order march
699,700,Can't log in to mi account. Keeps showing bloc...,Account Access,Low,UltraClean Vacuum,log mi account keep show block help delivery e...
696,697,Payment issue fr mi EcoBreeze AC. I was charge...,Billing Problem,High,EcoBreeze AC,payment issue fr mi ecobreeze ac charge twice ...
858,859,Can you tell me more about the RoboChef Blende...,General Inquiry,Medium,RoboChef Blender,tell robochef blender warranty available black


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
tfidf = TfidfVectorizer(max_features=300)

In [47]:
df['ticket_length'] = df['ticket_text_clean'].apply(lambda x: len(x.split()))

In [48]:
# !pip install TextBlob
from textblob import TextBlob

In [49]:
df['sentiment'] = df['ticket_text_clean'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [50]:
df.head()

,ticket_id,ticket_text,issue_type,urgency_level,product,ticket_text_clean,ticket_length,sentiment
0,1,Payment issue for my SmartWatch V2. I was unde...,Billing Problem,Medium,SmartWatch V2,payment issue smartwatch underbilled order,5,0.0
2,3,I ordered SoundWave 300 but got EcoBreeze AC i...,Wrong Item,Medium,SoundWave 300,order soundwave get ecobreeze ac instead order...,8,0.0
3,4,Facing installation issue with PhotoSnap Cam. ...,Installation Issue,Low,PhotoSnap Cam,face installation issue photosnap cam setup fa...,8,-0.5
5,6,Can you tell me more about the PhotoSnap Cam w...,General Inquiry,Medium,PhotoSnap Cam,tell photosnap cam warranty available red,6,0.2
6,7,is malfunction. It stopped working after just...,Product Defect,Low,EcoBreeze AC,malfunction stop work day,4,0.0


In [51]:
y_issue = df['issue_type']
y_urgency = df['urgency_level']

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [53]:
X_train_text, X_test_text, y_train_i, y_test_i = train_test_split(
    df['ticket_text_clean'], y_issue, test_size=0.2, random_state=42)# for issue 


In [54]:
X_train_i = tfidf.fit_transform(X_train_text)
X_test_i = tfidf.transform(X_test_text)

In [55]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': [None, 'balanced']
}
rf_base = RandomForestClassifier(random_state=42)

In [56]:
grid_search = GridSearchCV(estimator=rf_base, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='accuracy', 
                           n_jobs=-1, 
                           verbose=1)

In [57]:
grid_search.fit(X_train_i, y_train_i)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'max_depth': [3, 5, 10, None],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=1)

In [58]:
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'class_weight': None, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [59]:
best_rf_issue = grid_search.best_estimator_

In [60]:
y_pred_issue = best_rf_issue.predict(X_test_i)

In [61]:
print("classification report - issue type")
print(classification_report(y_test_i, y_pred_issue))

classification report - issue type
                    precision    recall  f1-score   support

    Account Access       1.00      1.00      1.00        23
   Billing Problem       1.00      1.00      1.00        19
   General Inquiry       1.00      1.00      1.00        25
Installation Issue       1.00      1.00      1.00        29
     Late Delivery       1.00      1.00      1.00        17
    Product Defect       1.00      1.00      1.00        30
        Wrong Item       1.00      1.00      1.00        23

          accuracy                           1.00       166
         macro avg       1.00      1.00      1.00       166
      weighted avg       1.00      1.00      1.00       166



In [62]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=300)),
    ('clf', RandomForestClassifier(random_state=42))
])

In [64]:
cv_scores = cross_val_score(pipeline, df['ticket_text_clean'], y_issue, cv=5)
print("Cross-Validation Accuracy Scores:", cv_scores)

Cross-Validation Accuracy Scores: [1. 1. 1. 1. 1.]


In [81]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [98]:
y_urgency = df['urgency_level']
X = df[['ticket_text_clean', 'sentiment', 'ticket_length', 'product']]

In [99]:
X_train, X_test, y_train_u, y_test_u = train_test_split(X, y_urgency, test_size=0.2, random_state=42, stratify=y_urgency)


In [100]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=2), 'ticket_text_clean'),
        ('num', StandardScaler(), ['sentiment', 'ticket_length'])
    ]
)

# Build the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])

In [101]:
param_grid = {
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [None, 3, 5, 10],
    'clf__min_samples_split': [2, 5],
    'clf__min_samples_leaf': [1, 2],
    'clf__class_weight': [None, 'balanced']
}

In [102]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=1)


In [103]:
grid_search.fit(X_train, y_train_u)


Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('text',
                                                                         TfidfVectorizer(max_features=3000,
                                                                                         min_df=2,
                                                                                         ngram_range=(1,
                                                                                                      2)),
                                                                         'ticket_text_clean'),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['sentiment',
                                                                          'ticket_length'])])),
                                       ('clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__class_weight': [None, 'balanced'],
                         'clf__max_depth': [None, 3, 5, 10],
                         'clf__min_samples_leaf': [1, 2],
                         'clf__min_samples_split': [2, 5],
                         'clf__n_estimators': [50, 100, 200]},
             scoring='f1_macro', verbose=1)

In [104]:
print("Best Parameters (Urgency):", grid_search.best_params_)

Best Parameters (Urgency): {'clf__class_weight': 'balanced', 'clf__max_depth': 3, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}


In [105]:
best_model = grid_search.best_estimator_
y_pred_u = best_model.predict(X_test)

print("Classification Report — Urgency Level:")
print(classification_report(y_test_u, y_pred_u))

Classification Report — Urgency Level:
              precision    recall  f1-score   support

        High       0.37      0.33      0.35        58
         Low       0.32      0.37      0.34        52
      Medium       0.34      0.34      0.34        56

    accuracy                           0.34       166
   macro avg       0.34      0.34      0.34       166
weighted avg       0.35      0.34      0.34       166



In [106]:
df['product'].value_counts()

product
RoboChef Blender     100
ProTab X1             89
EcoBreeze AC          86
PhotoSnap Cam         85
Vision LED TV         83
FitRun Treadmill      82
PowerMax Battery      80
SmartWatch V2         78
SoundWave 300         74
UltraClean Vacuum     69
Name: count, dtype: int64

In [116]:
product_list = [
    'RoboChef Blender', 'ProTab X1', 'EcoBreeze AC', 'PhotoSnap Cam', 'Vision LED TV',
    'FitRun Treadmill', 'PowerMax Battery', 'SmartWatch V2', 'SoundWave 300', 'UltraClean Vacuum'
]

product_list_lower = [p.lower() for p in product_list]

complaint_keywords = [
    'broken', 'late', 'error', 'not working', 'damaged', 'defective',
    'missing', 'delay', 'issue', 'problem', 'no response', 'malfunction',
    'not refunded', 'charged twice', 'wrong product', 'lost', 'blocked',
    'stopped working', 'login not working', 'installation issue', 'not here'
]

In [117]:
import re
from dateutil import parser

# Regex that captures formats like:
# "03 March", "13 April", "just 7 days", "after just 9 days", "expected by 04 March"
date_patterns = [
    r'\b(?:\d{1,2}[/-])?(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-]?\d{1,2}?\b',
    r'\b\d{1,2}\s+(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\b',
    r'\b(?:ordered|expected by|on)\s+\d{1,2}\s+(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b',
    r'after just \d+ days',
    r'just \d+ days',
    r'\d+ days late'
]

In [118]:
def extract_entities(text):
    entities = {'products': [], 'dates': [], 'complaints': []}
    if pd.isnull(text): return entities

    text_lower = text.lower()

    # Product match
    for product in product_list_lower:
        if product in text_lower:
            entities['products'].append(product)

    # Complaint keyword match (substring style)
    for kw in complaint_keywords:
        if kw in text_lower:
            entities['complaints'].append(kw)

    # Date match
    found_dates = []
    for pattern in date_patterns:
        matches = re.findall(pattern, text_lower)
        found_dates.extend(matches)

    entities['dates'] = found_dates

    return entities


In [127]:
df['entities'] = df['ticket_text'].apply(extract_entities)
df['entities'].head()

0    {'products': ['smartwatch v2'], 'dates': [], '...
2    {'products': ['ecobreeze ac', 'soundwave 300']...
3    {'products': ['photosnap cam'], 'dates': [], '...
5    {'products': ['photosnap cam'], 'dates': [], '...
6    {'products': [], 'dates': ['after just 7 days'...
Name: entities, dtype: object

In [131]:
df['entities'].apply(lambda x: x['complaints']).head(20)

0                                      [issue]
2                                           []
3                  [issue, installation issue]
5                                           []
6               [malfunction, stopped working]
7     [issue, no response, installation issue]
11                                  [not here]
13                  [issue, no response, lost]
14                                      [late]
15                                      [late]
16                [no response, wrong product]
17                             [wrong product]
18                                     [issue]
19            [not working, login not working]
20                           [stopped working]
21          [issue, no response, not refunded]
22                  [damaged, stopped working]
23                                   [blocked]
26                 [issue, installation issue]
27                                          []
Name: entities, dtype: object

In [136]:
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text).lower().strip()


def process_ticket(ticket_text):
    # Clean text
    text_clean = clean_text(ticket_text)

    # Extract numerical features
    sentiment = TextBlob(text_clean).sentiment.polarity
    ticket_length = len(text_clean.split())

    # Prepare input for issue_type
    X_issue = tfidf.transform([text_clean])
    predicted_issue = best_rf_issue.predict(X_issue)[0]

    # Prepare input for urgency_level (use DataFrame for pipeline)
    input_df = pd.DataFrame([{
        'ticket_text_clean': text_clean,
        'sentiment': sentiment,
        'ticket_length': ticket_length,
        'product': None  # You can optionally add logic to map product
    }])
    predicted_urgency = best_model.predict(input_df)[0]

    # Entity extraction
    entities = extract_entities(ticket_text)

    # Final result
    return {
        'predicted_issue_type': predicted_issue,
        'predicted_urgency_level': predicted_urgency,
        'entities': entities
    }


In [137]:
ticket = "Order #30903 for Vision LED TV is 13 days late. Ordered on 03 March. No response from support."
output = process_ticket(ticket)

from pprint import pprint
pprint(output)

{'entities': {'complaints': ['late', 'no response'],
              'dates': ['03 march', '13 days late'],
              'products': ['vision led tv']},
 'predicted_issue_type': 'Late Delivery',
 'predicted_urgency_level': 'High'}


In [138]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [139]:
import joblib

joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

joblib.dump(best_rf_issue, 'rf_issue_model.pkl')

joblib.dump(best_model, 'urgency_model.pkl')


['urgency_model.pkl']